## Upload do arquivo ENEM Microdados no RDS da AWS

# Download do arquivo

In [1]:
# importando as bibliotecas a serem utilizadas 

import zipfile
import requests
from io import BytesIO


In [2]:
# criando o diretório para armazenar o conteúdo do ENEM 2019
import os

os.makedirs('./enem2020', exist_ok = True)

In [3]:
# definindo a url para download

URL = 'https://download.inep.gov.br/microdados/microdados_enem_2020.zip'

# fazendo o download do conteúdo

fileContent = BytesIO(
    requests.get(URL).content
    )

# extraindo o conteúdo do zipfile e armazenando na pasta ENEM 2019

myzip = zipfile.ZipFile(fileContent)
myzip.extractall("./enem2020")

## Conexão com o RDS do servidor AWS e upload do arquivo

In [4]:
import boto3

# Criar um cliente para interagir com o AWS S3
s3_client = boto3.client('s3')

s3_client.upload_file('enem2020/DADOS/MICRODADOS_ENEM_2020.csv',
                      'trab-pratico-igti',
                      'raw_data/enem/microdados_enem_2020.csv')

## Script  para converter os dados csv em parquet via Glue (Spark Script)

In [ ]:
import pyspark
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import job

## @params: [EnemCSVConvert]
args = getResolvedOptions(sys.argv, ['EnemCSVConvert'])

sc = SparkContext()
glueContext = GlueContext(sc)
spark = glueContext.spark_session
job = Job(glueContext)
job.init(args['EnemCSVConvert'], args)

## Ler os dados do enem 2020
enem = (
    spark
    .read
    .format("csv")
    .option("header", True)
    .option("inferSchema", True)
    .option("delimiter", ";")
    .load("s3://trab-pratico-igti/raw_data/enem/")
)

## Escrever no DataLake os dados em formato parquet
(
    enem
    .write
    .mode("overwrite")
    .format("parquet")
    .partitionBy("year")
    .save("s3://trab-pratico-igti/staging/enem/")
)




## Scripts consultas SQL

Pergunta 6 (TP)

Sobre o ENEM 2020, qual é a média da nota em matemática apenas para as alunas do sexo Feminino?
Grupo de escolhas da pergunta

523,21.

489,18.

500,69.

497,86.



------

SELECT tp_sexo, avg(nu_nota_mt)
FROM enem
GROUP BY tp_sexo

OR

SELECT avg(nu_nota_mt)
FROM enem
WHERE tp_sexo = 'F'

Resposta = 500,69

Pergunta 7 (TP)

Qual é a média da nota em Ciências Humanas para os alunos do sexo masculino que estudaram numa escola no estado de São Paulo?
Grupo de escolhas da pergunta

553,68.

519,97.

481,12.

496,13.

-----------------------------------------------

SELECT tp_sexo, AVG(nu_nota_ch) as media_nota_humanas
FROM enem
WHERE sg_uf_esc = 'SP'
GROUP BY tp_sexo


OR


SELECT AVG(nu_nota_ch) as media_nota_humanas
FROM enem
WHERE sg_uf_esc = 'SP' and tp_sexo = 'M'


Resposta = 553,68

Pergunta 8 (TP)

Qual é a cidade (município da escola) que possui a MAIOR nota em Ciências Naturais?
Grupo de escolhas da pergunta

Ouro Preto.

Montes Claros.

Ipatinga.

Governador Valadares.

-------------------------------------------------

SELECT no_municipio_esc, MAX(nu_nota_cn) as Nota_Ciencias_Naturais 
FROM enem
WHERE no_municipio_esc IN ('Ouro Preto', 'Montes Claros', 'Ipatinga', 'Governador Valadares')
GROUP BY no_municipio_esc
ORDER BY Nota_Ciencias_Naturais  DESC

Resposta = Ipatinga

PERGUNTA 9 (TP)

Qual é a média da nota em Ciências Humanas dos alunos que estudaram numa escola na cidade de Natal?
Grupo de escolhas da pergunta

589,45

523,64

536,17

548,18
 
--------------------------------------------

SELECT AVG(nu_nota_ch) as Media_Nota_Ciencias_Humanas
FROM enem
WHERE no_municipio_esc = 'Natal'

Resposta = 548,18

PERGUNTA 10 (TP)

Qual é o município (município da escola) que obteve a maior nota MÉDIA em matemática?
Grupo de escolhas da pergunta

Biquinhas

Dolcinópolis

Novo Xingu

Novo Itacolomi
 
--------------------------------------------------

SELECT no_municipio_esc, AVG(nu_nota_mt) as Media_Nota_Matematica
FROM enem
GROUP BY no_municipio_esc
ORDER BY Media_Nota_Matematica DESC
LIMIT 1

OR

SELECT no_municipio_esc 
FROM
    (SELECT no_municipio_esc, AVG(nu_nota_mt) as Media_Nota_Matematica
    FROM enem
    GROUP BY no_municipio_esc
    ORDER BY Media_Nota_Matematica DESC
    LIMIT 1)


Resposta = Novo Xingu

PERGUNTA 11 (TP)

Quantas pessoas que estudaram numa escola no estado do Rio de Janeiro do sexo masculino obtiveram nota em matemática maior do que 600?
Grupo de escolhas da pergunta

5847

9684

10574

8533
 
-------------------------------------------

SELECT COUNT(*)
FROM enem
WHERE tp_sexo = 'M' 
AND sg_uf_esc = 'RJ' 
AND nu_nota_mt > 600

Resposta : 8578????

PREGUNTA 12 (TP)

Quantas pessoas que estudaram numa escola em Recife fizeram a prova do ENEM nessa mesma cidade?
Grupo de escolhas da pergunta

11827

9619

10474

12064
 
---------------------------------------------

SELECT COUNT(*)
FROM enem
WHERE no_municipio_esc = 'Recife'
AND no_municipio_prova = 'Recife'


Resposta = 9619

PERGUNTA 13 (TP)

Quantas mulheres que moram sozinhas, estudaram numa escola no estado de SP fizeram a prova de Ciências Humanas do ENEM em 2020?
Grupo de escolhas da pergunta

256

45

90

155
 
---------------------------------------------

SELECT COUNT(*)
FROM enem
WHERE q005 = 1
AND sg_uf_esc = 'SP'
AND tp_presenca_ch = 1
AND tp_sexo = 'F'

Resposta : 65???

Para mulheres E homens -> 155

PERGUNTA 14 (TP)


Qual é a nota média em Ciências Humanas dos alunos que estudaram numa escola no estado de Santa Catarina e possuem PELO MENOS 1 banheiro em casa?
Grupo de escolhas da pergunta

497,53

512,34

436,52

545,04
 
---------------------------

SELECT AVG(nu_nota_ch)
FROM enem
WHERE sg_uf_esc = 'SC'
AND q008 <> 'A'


Resposta = 545,04

PERGUNTA 15 (TP)

Qual é a nota média em matemática dos alunos cuja mãe possui PELO MENOS o ensino superior completo, do sexo feminino que estudaram numa escola em Belo Horizonte?
Grupo de escolhas da pergunta

639,27

667,13

684,35

578,68

------------------------------

SELECT AVG(nu_nota_mt)
FROM enem
WHERE no_municipio_esc = 'Belo Horizonte'
AND tp_sexo = 'F'
AND q002 in ('F','G')


Resposta = 667,13